# importing dependencies

In [88]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [89]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

# Data pre-processing

## Data collecting

In [91]:
df = pd.read_csv('train.csv')

## Data exploration

In [92]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [93]:
df.shape

(20800, 5)

In [94]:
df.dtypes

id         int64
title     object
author    object
text      object
label      int64
dtype: object

## Data Cleaning

In [95]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [96]:
df.duplicated().sum()

0

In [97]:
df.fillna('', inplace=True)

In [98]:
df.isna().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

## Feature Engineering

In [99]:
df['content'] = df['author'] + ' ' + df['title']

In [100]:
df.columns

Index(['id', 'title', 'author', 'text', 'label', 'content'], dtype='object')

##  d

In [101]:
X = df.drop('label', axis=1)
Y = df['label']

In [102]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

## Steaming
### steaming is the process of reducing a word to it's root word

#### EX: actor, actress, acting ===> act


In [103]:
p_stm = PorterStemmer()

In [104]:
def steaming(content):
  steammed_content= re.sub('[^a-zA-Z]', ' ', content) # remove any thing that is not between a-z
  steammed_content = steammed_content.lower() # change everything to lowercase
  steammed_content = steammed_content.split() # split the word to lists
  steammed_content = [p_stm.stem(word) for word in steammed_content if not word in stopwords.words('english') ]# if the word is not a stopword steam it
  steammed_content = ' '.join(steammed_content)
  return steammed_content

In [105]:
s = 'Hello world '
n = s.split()
print(n)

['Hello', 'world']


In [106]:
d = ' '.join(n)
print(d)

Hello world


In [107]:
df['content'] = df['content'].apply(steaming)

In [108]:
print(df['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


In [109]:
df['id'].values

array([    0,     1,     2, ..., 20797, 20798, 20799])

In [110]:
X = df['content'].values
Y = df['label'].values

In [111]:
print(X)
print(Y)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michael j de la merc rachel abram maci said receiv takeov approach hudson bay new york time'
 'alex ansari nato russia hold parallel exercis balkan'
 'david swanson keep f aliv']
[1 0 1 ... 0 1 1]


In [112]:
Y.shape

(20800,)

In [113]:
X.shape

(20800,)

## changing text data to numerical data

In [114]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [115]:
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

# Train_Test

## split the data

In [116]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=10)

## Model

In [117]:
lgr = LogisticRegression()

In [118]:
lgr.fit(X_train, Y_train)

LogisticRegression()

## Model accuracy

### Training accuracy

In [119]:
X_train_predict = lgr.predict(X_train)
X_train_acc = accuracy_score(X_train_predict, Y_train)

In [120]:
print(X_train_acc)

0.9861778846153846


### Testing accuracy

In [121]:
X_test_predict = lgr.predict(X_test)
X_test_acc = accuracy_score(X_test_predict, Y_test)

In [122]:
print(X_test_acc)

0.9798076923076923


In [184]:
X_new = X_test[1500]
print(Y_test[1500])

1


In [185]:
predict = lgr.predict(X_new)
print(predict)

[1]
